# PFQ - Aula 29 - Identificação de ciclos com indicador da média

In [9]:
import yfinance as yf
import pandas as pd
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

In [15]:
# Captura os dados do Yahoo Finance
ticker = "^GSPC"
start_date = "2014-01-01"
end_date = "2024-12-31"

data = yf.download(ticker, start=start_date, end=end_date, progress=False)

# Calcula a média móvel de 52 períodos
data['SMA_52'] = data['Adj Close'].rolling(window=52).mean()

# Calcula o número de dias acumulados acima e abaixo da média móvel
data['Days_Above'] = 0
data['Days_Below'] = 0

above_count = 0
below_count = 0

for i in range(len(data)):
    if data['Adj Close'].iloc[i] > data['SMA_52'].iloc[i]:
        above_count += 1
        below_count = 0
        data['Days_Above'].iloc[i] = above_count
    elif data['Adj Close'].iloc[i] < data['SMA_52'].iloc[i]:
        below_count += 1
        above_count = 0
        data['Days_Below'].iloc[i] = below_count
    else:
        # Reseta os contadores se Adj Close for igual à SMA
        above_count = 0
        below_count = 0

# Cria dois subplots
fig = sp.make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=("Adj Close & SMA 52", "Days Above/Below SMA 52"),
)

# Subplot 1: Adj Close e SMA
fig.add_trace(
    go.Scatter(x=data.index, y=data['Adj Close'], mode='lines', name='Adj Close'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=data.index, y=data['SMA_52'], mode='lines', name='SMA 52', line=dict(dash='dot')),
    row=1, col=1
)

# Subplot 2: Indicadores de dias acima/abaixo
fig.add_trace(
    go.Scatter(x=data.index, y=data['Days_Above'], mode='lines', name='Days Above SMA'),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=data.index, y=data['Days_Below'], mode='lines', name='Days Below SMA'),
    row=2, col=1
)

# Layout do gráfico
fig.update_layout(
    title=f"{ticker} Analysis: Adj Close & Days Above/Below SMA 52",
    height=700, width = 1000,
    showlegend=True,
)

# Ajusta os títulos dos eixos
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Days Count", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

# Exibe o gráfico
fig.show()

In [16]:
fig = sp.make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=("Adj Close (Dynamic Color) & SMA 52", "Days Above/Below SMA 52"),
)

# Subplot 1: Adj Close com cores dinâmicas
fig.add_trace(
    go.Scatter(
        x=data.index,
        y=np.where(data['Days_Above'] >= 36, data['Adj Close'], None),  # Vermelho para Days_Above >= 36
        name="Adj Close (>=36 Days Above SMA)",
        line=dict(color="red"),
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=data.index,
        y=np.where(data['Days_Above'] < 36, data['Adj Close'], None),  # Azul para Days_Above < 36
        name="Adj Close (<36 Days Above SMA)",
        line=dict(color="blue"),
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=data.index,
        y=data['SMA_52'],
        mode='lines',
        name='SMA 52',
        line=dict(dash='dot', color='green'),
    ),
    row=1, col=1
)

# Subplot 2: Indicadores de dias acima/abaixo
fig.add_trace(
    go.Scatter(x=data.index, y=data['Days_Above'], mode='lines', name='Days Above SMA', line=dict(color='blue')),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=data.index, y=data['Days_Below'], mode='lines', name='Days Below SMA', line=dict(color='violet')),
    row=2, col=1
)

# Layout do gráfico
fig.update_layout(
    title=f"{ticker} Analysis: Adj Close with Dynamic Color & Days Above/Below SMA 52",
    height=700, width = 1000,
    showlegend=True,
)

# Ajusta os títulos dos eixos
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Days Count", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

# Exibe o gráfico
fig.show()